In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import hnswlib

import streamlit as st

path = 'D:\Ahmed\GitHub projects\personalised-recommendation-system\All Appliances.csv'
model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
def read_data(path):
    data = pd.read_csv(path)
    data['combined_features'] = data['name'] + ' ' + \
        data['main_category'] + ' '+data['sub_category']
    return data

In [3]:
selected_category = 'appliances'
search_query = 'Kettle'
data = read_data(path)
print('read data done')
selected_data = data[data['main_category'] == selected_category]
print('select data done')

read data done
select data done


In [4]:
sentences = selected_data['combined_features'].to_list()


In [5]:
# embeddings = model.encode(sentences)

In [6]:
# type(embeddings)

In [7]:
# np.save('data.npy', embeddings) # save

In [8]:
embeddings = np.load('data.npy') # load


In [10]:
print("encoding data done")
dim = embeddings.shape[1]  # Dimension of embeddings
num_elements = selected_data.shape[0]
print('dim:', dim)
print('num elements:', num_elements)

encoding data done
dim: 384
num elements: 9576


In [ ]:
p = hnswlib.Index(space='cosine', dim=dim)
p.init_index(max_elements=10000, ef_construction=200, M=16)
p.add_items(embeddings)
p.set_ef(50) # ef should always be > k

: 

In [25]:
# Query HNSW index for most similar sentence
new_sentence = "mixer"
new_embedding = model.encode([new_sentence])

# Fetch k neighbors
labels, distances = p.knn_query(new_embedding, k=5)
labels = labels.flatten()
# print("Most similar sentence is: ", sentences[labels[0]])

Most similar sentence is:  FINALIZE 7-Speed Electric Beater Hand Blender Mixer for Cake Mixing ,Baking with 4 Pieces Stainless Hooks Blender and High... appliances All Appliances


In [41]:
print(selected_data.iloc[labels[0]])

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,combined_features
9534,FINALIZE 7-Speed Electric Beater Hand Blender ...,appliances,All Appliances,https://m.media-amazon.com/images/W/IMAGERENDE...,https://www.amazon.in/FINALIZE-Electric-Stainl...,3.9,43,₹598,"₹1,999",FINALIZE 7-Speed Electric Beater Hand Blender ...
6702,R k g Enterprises Scarlett Super Hand Mixer Wi...,appliances,All Appliances,https://m.media-amazon.com/images/I/61ovt+M0I3...,https://www.amazon.in/Scarlett-Original-Multic...,NaN,NaN,₹525,"₹1,302",R k g Enterprises Scarlett Super Hand Mixer Wi...
7223,R k g Enterprises Scarlett Super Hand Mixer Wi...,appliances,All Appliances,https://m.media-amazon.com/images/I/61ovt+M0I3...,https://www.amazon.in/Scarlett-Original-Multic...,NaN,NaN,₹525,"₹1,302",R k g Enterprises Scarlett Super Hand Mixer Wi...
6533,Arth-Enterprises 7 Speed Hand Mixer With 4 Pie...,appliances,All Appliances,https://m.media-amazon.com/images/I/41y-ZfKPMn...,https://www.amazon.in/Stainless-Blender-Kitche...,3.5,23,₹499,"₹1,299",Arth-Enterprises 7 Speed Hand Mixer With 4 Pie...
9513,Finalize Hand Mixer Beater Blender for Cake Ba...,appliances,All Appliances,https://m.media-amazon.com/images/I/61BZOhtSRP...,https://www.amazon.in/Finalize-Blender-Electri...,3.9,248,₹649,"₹1,499",Finalize Hand Mixer Beater Blender for Cake Ba...


In [1]:
import hnswlib
import numpy as np
from sentence_transformers import SentenceTransformer
import time

# Initialize Sentence Transformer Model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Sentences to embed and store
sentences = ['This is an example sentence', 'This is another one']

# Create embeddings
embeddings = model.encode(sentences)

# Dimension of our vector space
dimension = embeddings.shape[1]

# Create a new index
p = hnswlib.Index(space='cosine', dim=dimension)

# Initialize an index - the maximum number of elements should be known beforehand
p.init_index(max_elements=10000, ef_construction=200, M=16)

# Element insertion (can be called several times)
p.add_items(embeddings)

# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

# Query HNSW index for most similar sentence
new_sentence = "A new sentence similar to the previous ones"
new_embedding = model.encode([new_sentence])

# Fetch k neighbors
labels, distances = p.knn_query(new_embedding, k=1)

print("Most similar sentence is: ", sentences[labels[0][0]])

# Measure the speed
start_time = time.time()

# Repeat the search process to get a fair measure
for _ in range(1000):
    labels, distances = p.knn_query(new_embedding, k=1)

end_time = time.time()
elapsed_time = end_time - start_time

print("Elapsed time for 1000 searches: ", elapsed_time)

c:\Users\ahmed\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



Most similar sentence is:  This is an example sentence
Elapsed time for 1000 searches:  0.006000995635986328
